I used a dataset about student's alcohol consumption
I chose the math table
The explanations of the abreviations of each variable are on this page
https://www.kaggle.com/uciml/student-alcohol-consumption


Since I thought this was good quality data, it's best to use the whole dataset. I tested 7 types of classifiers.

I combined two variables of alcohol consumption during the whole week for students (weekend+workday). These variables are Walc=weekend alcohol and Dalc=workday alcohol. I transformed the pred class to values of 0 and 1
By reading the page on Kaggle, for one var of alcohol consumption (Walc OR Dalc), 4 or 5 is high or very high.
1: I chose that high consumption of alcohol was equal to or higher than 7 (Walc+Dalc) 
0: consumption of alcohol was lower than 7 (Walc+Dalc)

The verification of the best algorithm was verified through the confusion matrix, but I especially looked at the accuracy score.

In [ ]:
# Loading the packages

install.packages('tidyverse')
install.packages('data.table')
install.packages('rpart')
install.packages('rpart.plot')
install.packages('tree')
install.packages('forestmangr')
install.packages('caret')
install.packages('e1071')
install.packages('ggplot2')
install.packages('ROCR')
install.packages('lattice')

rm(list=ls())

library(tidyverse)
library(data.table)
library(rpart)
library(rpart.plot)
library(tree)
library(forestmangr)
library(caret)
library(e1071)
library(nnet) 
library(ggplot2)
library(ROCR)
library(lattice);
library(latticeExtra);

In [ ]:
# Loading the dataset
#setwd(dir = "/Users/ellie/Documents/Cog-M1/big data")
Table<- read.csv('student-mat.csv')

In [ ]:
# Averaging the grades to get one parameter and not 3 :
Col_grades <- (Table$G1 + Table$G2 + Table$G3)/3
Table$Grades <- cbind(Col_grades)
Table <- subset(Table,select=-G1)
Table <- subset(Table,select=-G2)
Table <- subset(Table,select=-G3)

# Summing workday and weekend alcohol consumption :
Col_alc <- Table$Walc + Table$Dalc
Table$Alcohol <- cbind(Col_alc)
Table <- subset(Table,select=-Walc)
Table <- subset(Table,select=-Dalc)


In [ ]:
#Transforming into binary values (high=1 vs moderate/low=0 consumption)
#Replacing the high values of consumption (>=7) by 1 and the lower than 7 by 0.
Table$Alcohol[Table$Alcohol<7] <- 0
Table$Alcohol[Table$Alcohol>=7] <- 1
View(Table)

In [ ]:
# Showing the summary and number of high and moderate/low
table(Table$Alcohol)
summary(Table$Alcohol)
sapply(Table, class)
dim(Table)
pie(table(Table$Alcohol), main="Class repartition")

In [ ]:
#Shuffling the dataset
Table1 <- Table[sample(nrow(Table)),]

# Seperating into training and testing
training.samples <- Table1$Alcohol %>% createDataPartition(p = 0.75, list = FALSE)
t_train  <- Table1[training.samples, ]
t_test <- Table1[-training.samples, ]
dim(t_train)
dim(t_test)

I'll first build tree type of model to see the variables that we can plot later on

In [ ]:
# Build the models
# Creating and visualizing tree1 with tree
tree1 <- tree(Alcohol ~., data = t_train)
plot(tree1)
text(tree1, pretty=0)

In [ ]:
# Creating and visualizing tree2 with rpart tree
tree2 <- rpart(Alcohol~., t_train)
prp(tree2, type=4, extra=0, box.col=c("orange","skyblue","pink","grey")[tree2$frame$yval])

In [ ]:
# Applying the tree model on t_test
test_tree1 <- predict(tree1, t_test, probability=TRUE)
test_tree2 <- predict(tree2, t_test, probability=TRUE)
#predicted
test_tree1 <- data.frame(test_tree1)
test_tree2 <- data.frame(test_tree2)
#reference
Col_alc_test <- data.frame(t_test$Alcohol)

# Rounding the values of the predicted data to get 0 or 1
test_tree1 <- round_df(test_tree1, digits=0)
data_tree1 <- data.frame(Col_alc_test, test_tree1)
test_tree2 <- round_df(test_tree2, digits=0)
data_tree2 <- data.frame(Col_alc_test, test_tree2)

In [ ]:
#Confusion matrix of tree and rpart
table(data_tree1)
table(data_tree2)

Let's plot the variables of the trees to see how visually correlated they are to alcohol consumption
They seem to be (in both trees): the sex, the going out degree and the number of absences.

In [ ]:
# Quick plotting using the data of the tree classifier
boxplot(Table$goout~Table$Alcohol, data=Table,col=c("tomato","darkturquoise"), 
        main="Alcohol according to going out degree",xlab="Alcohol consumption",ylab="Going out degree")

-> There's definitely a link between going out and high alcohol consumption

In [ ]:
qplot(Table$Alcohol, Table$goout, data=Table, color=Table$sex,
      main="Alcohol consumption according to going out and Sex",xlab="Alcohol consumption",ylab="Going out degree",)+geom_jitter(height=0.1)

-> There's a lot more boys in the high alcohol consumption

In [ ]:
qplot(Alcohol, goout, data=Table, 
      main="Going out degree according to alcohol consumption", xlab="Alcohol consumption", ylab="Go out degree") + geom_smooth(method = "lm", formula=y~x)

In [ ]:
qplot(Alcohol, absences, data=Table, 
      main="Number of absences according to alcohol consumption", xlab="Alcohol consumption", ylab="Nb of absences") + geom_smooth(method = "lm", formula=y~x)

On the tree, the second most predictable parameter with alcohol is the number of absences

In [ ]:
qplot(Table$goout, Table$absences, data=Table, color=Table$Alcohol,
      main="Alcohol consumption according to going out and Absences",xlab="Go out degree",ylab="Nb absences",)+geom_jitter(height=2)

-> Here we have a lot more students with high alcohol consumption if there's a lot of going out, the absence isn't visible.

In [ ]:
# Setting the reference
reference <- t_test$Alcohol

In [ ]:
#10-fold cross validation
train_control <- trainControl(method = "cv", number = 10)
metric <- "Accuracy"

In [ ]:
#TREE Model
data_tree1 <- as.numeric(test_tree1>0.5)
confusionMatrix(as.factor(data_tree1), as.factor(reference))

In [ ]:
#RPART Model (trained before plot)
data_tree2 <- as.numeric(test_tree2>0.5)
confusionMatrix(as.factor(data_tree2), as.factor(reference))

In [ ]:
# GLM Model
model_glm <- glm(t_train$Alcohol~.,data=t_train, family=binomial(link="logit"))
test_glm <- predict(model_glm, t_test, type = "response")
data_glm = as.numeric(test_glm>0.5)
confusionMatrix(as.factor(data_glm), as.factor(reference))

In [ ]:
# KNN Model
model_knn <- train(Alcohol~., data=t_train, method="knn", trControl=train_control)
test_knn <- predict(model_knn, t_test, type = "raw")
data_knn = as.numeric(test_knn>0.5)
confusionMatrix(as.factor(data_knn), as.factor(reference))

In [ ]:
# SVM Model
model_svm <- svm(Alcohol~., data=t_train, method = 'svm',
                 trControl = train_control)
test_svm <- predict(model_svm, t_test, type = "raw")
data_svm <- as.numeric(test_svm>0.5)
cm_svm <- confusionMatrix(as.factor(data_svm), as.factor(reference))
print(cm_svm)

In [ ]:
# NNET Model
model_nnet <- nnet(Alcohol ~ ., data=t_train, size=2)
test_nnet <- predict(model_nnet, t_test, type = "raw")
data_nnet <- as.numeric(test_nnet>0.5)
confusionMatrix(as.factor(data_nnet), as.factor(reference))

In [ ]:
# Random forest Model
model_rf <- train(Alcohol~., data=t_train, method="rf", trControl=train_control)
test_rf <- predict(model_rf, t_test, type = "raw")
data_rf <- as.numeric(test_rf>0.5)
confusionMatrix(as.factor(data_rf), as.factor(reference))

ACCURACY FOR EACH CLASSIFIER

Tree: .8571

Rpart: .8673

GLM: .8878

KNN: .9184

SVM: .9184

Nnet: .9184

RF: .9082


=> NNET, SVM and KNN predicted all values as 0 and got the best accuracy score

----
BONUS : Testing one of the best classifier to other methods of cross validation


In [ ]:

# Repeated 10 fold cross validation
train_control1 <- trainControl(method = "repeatedcv", 
                              number = 10, repeats = 3)
#re-training
model_svm1 <- svm(Alcohol~., data=t_train, method = 'svm',
                 trControl = train_control1)
test_svm1 <- predict(model_svm1, t_test, type = "raw")
data_svm1 <- as.numeric(test_svm1>0.5)
cm_svm1<-confusionMatrix(as.factor(data_svm1), as.factor(reference))

=> This gives us the same exact result of accuracy 
But not the same result of confusion matrix !

In [ ]:
cm_svm
cm_svm1

In [ ]:
#Leave one out cross validation
train_control2 <- trainControl(method = "LOOCV")
#re-training
model_svm2 <- svm(Alcohol~., data=t_train, method = 'svm',
                 trControl = train_control2)
test_svm2 <- predict(model_svm2, t_test, type = "raw")
data_svm2 <- as.numeric(test_svm2>0.5)
cm_svm2 <- confusionMatrix(as.factor(data_svm2), as.factor(reference))


Same results for this one too.

In [ ]:
cm_svm
cm_svm1
cm_svm2

In [ ]:
# Best models
print(model_svm)
print(model_nnet)
print(model_knn)